In [4]:
# Install libraries required
%pip install --upgrade --quiet langchain langchain-community langchain-openai faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# import libraries required
import os
from langchain.chains import create_sql_query_chain # type: ignore
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool # type: ignore
from langchain_community.utilities import SQLDatabase # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
from langchain_core.prompts import PromptTemplate # type: ignore
from langchain_core.runnables import RunnablePassthrough # type: ignore
from langchain_openai import ChatOpenAI # type: ignore
from operator import itemgetter

In [6]:
# connect to chat gpt
os.environ["OPENAI_API_KEY"] = ''

In [7]:
# Connect to the database and test the connection
db = SQLDatabase.from_uri("sqlite:///spotify.db")

# Print the type of database dialect being used (SQLite3 in this case)
print(db.dialect)

# Print the names of the tables that can be used in this database
print(db.get_usable_table_names())

# Run a test query to select the first row from the 'top_streams' table
db.run("SELECT * FROM top_streams LIMIT 1;")

# Print the column names of the 'top_streams' table
print("\nColumn names:")
db.run("SELECT name FROM PRAGMA_TABLE_INFO('top_streams');")

sqlite
['top_streams']

Column names:


"[('Track',), ('Album Name',), ('Artist',), ('Release Date',), ('ISRC',), ('All Time Rank',), ('Track Score',), ('Spotify Streams',), ('Spotify Playlist Count',), ('Spotify Playlist Reach',), ('Spotify Popularity',), ('YouTube Views',), ('YouTube Likes',), ('TikTok Posts',), ('TikTok Likes',), ('TikTok Views',), ('YouTube Playlist Reach',), ('Apple Music Playlist Count',), ('AirPlay Spins',), ('SiriusXM Spins',), ('Deezer Playlist Count',), ('Deezer Playlist Reach',), ('Amazon Playlist Count',), ('Pandora Streams',), ('Pandora Track Stations',), ('Soundcloud Streams',), ('Shazam Counts',), ('TIDAL Popularity',), ('Explicit Track',)]"

In [8]:
# choose model
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
# Create a tool to execute queries on the connected database
execute_query = QuerySQLDataBaseTool(db=db)

# Set up a process that uses a large language model (LLM) to write SQL queries for the database
write_query = create_sql_query_chain(llm, db)

# Chain the two processes together: the first part writes the query, and the second part executes it
chain = write_query | execute_query

# Use the chained process to answer the question "How many tracks are there?" by invoking the query
chain.invoke({"question": "How many tracks are there"})

'[(4600,)]'

In [10]:
# prompt
answer_prompt = PromptTemplate.from_template(
"""You are an agent that uses data from a Spotify Database about the most streamed songs on spotify in 2024.
Here is a description of each column;


Track Name: Name of the song.
Album Name: Name of the album the song belongs to.
Artist: Name of the artist(s) of the song.
Release Date: Date when the song was released.
ISRC: International Standard Recording Code for the song.
All Time Rank: Ranking of the song based on its all-time popularity.
Track Score: Score assigned to the track based on various factors.
Spotify Streams: Total number of streams on Spotify.
Spotify Playlist Count: Number of Spotify playlists the song is included in.
Spotify Playlist Reach: Reach of the song across Spotify playlists.
Spotify Popularity: Popularity score of the song on Spotify.
YouTube Views: Total views of the song's official video on YouTube.
YouTube Likes: Total likes on the song's official video on YouTube.
TikTok Posts: Number of TikTok posts featuring the song.
TikTok Likes: Total likes on TikTok posts featuring the song.
TikTok Views: Total views on TikTok posts featuring the song.
YouTube Playlist Reach: Reach of the song across YouTube playlists.
Apple Music Playlist Count: Number of Apple Music playlists the song is included in.
AirPlay Spins: Number of times the song has been played on radio stations.
SiriusXM Spins: Number of times the song has been played on SiriusXM.
Deezer Playlist Count: Number of Deezer playlists the song is included in.
Deezer Playlist Reach: Reach of the song across Deezer playlists.
Amazon Playlist Count: Number of Amazon Music playlists the song is included in.
Pandora Streams: Total number of streams on Pandora.
Pandora Track Stations: Number of Pandora stations featuring the song.
Soundcloud Streams: Total number of streams on Soundcloud.
Shazam Counts: Total number of times the song has been Shazamed.
TIDAL Popularity: Popularity score of the song on TIDAL.
Explicit Track: Indicates whether the song contains explicit content.
    
    
Given the following user question, corresponding SQL query, and SQL result, answer the user question in a human friendly tone. 
Take each column description into consideration when making a decision.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [11]:

# Create a process that passes through the input and assigns the query using the write_query chain
chain = (
    RunnablePassthrough.assign(query=write_query)
    
    # Assign the result of executing the query to the output
    .assign(result=itemgetter("query") | execute_query)
    
    # Pass the result to the answer prompt
    | answer_prompt
    
    # Use the large language model (LLM) to generate a response
    | llm
    
    # Parse the output into a string format
    | StrOutputParser()
)

In [12]:
# Run the entire chained process to answer the question 
chain.invoke({"question": "How many artists are there"})

'There are a total of 2000 artists in the Spotify Database based on the most streamed songs in 2024.'

In [15]:
# Ask the user for their question, then run the entire chained process to answer it
chain.invoke({"question": input("What is your question?")})

'The most liked artist based on YouTube likes is Vishal Mishra, with a total of 998,592 likes on their official videos.'